CMD commands =>

In [ ]:
valeria.lupanova@bd-master:~$ hadoop fs -ls /user/valeria.lupanova
valeria.lupanova@bd-master:~$ hadoop fs -mkdir /user/valeria.lupanova/lab03/
valeria.lupanova@bd-master:~$ hadoop fs -mkdir /user/valeria.lupanova/lab03/input
valeria.lupanova@bd-master:~$ hadoop fs -mkdir /user/valeria.lupanova/lab03/output
valeria.lupanova@bd-master:~$ hadoop fs -cat /user/valeria.lupanova/lab03/input/part-00000 | head -n100
    
valeria.lupanova@bd-master:~$ beeline jdbc:hive2://bd-node1.newprolab.com:10000 -n valeria.lupanova

password VD3H6ehz

use valeria_lupanova;
show tables;

Table for analysing data =>

In [ ]:
create external table valeria_lupanova.raw_data
(uid string comment 'Уникальный идентификатор пользователя', 
 ts string comment 'Отметка времен в UNIX формате', 
 domain string comment 'Экранированный URL')
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
LOCATION 'hdfs://bd-master.newprolab.com:8020/user/valeria.lupanova/lab03/input';

In [ ]:
msck repair table valeria_lupanova.raw_data;
select * from valeria_lupanova.raw_data limit 100;

Table for resulting data =>

In [ ]:
create external table valeria_lupanova.valeria_lupanova 
(uid bigint, 
 user_cat1_flag bigint, 
 user_cat2_flag bigint,  
 user_cat3_flag bigint, 
 user_cat4_flag bigint)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
LOCATION 'hdfs://bd-master.newprolab.com:8020/user/valeria.lupanova/lab03/output';

In [ ]:
+-------------------+
|     tab_name      |
+-------------------+
| input_data        |
| valeria_lupanova  |
+-------------------+

In [ ]:
INSERT OVERWRITE DIRECTORY 'hdfs://bd-master.newprolab.com:8020/user/valeria.lupanova/lab03/output'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
select distinct a.uid as uid, 
case when a.domain in ('cars.ru', 'avto-russia.ru', 'bmwclub.ru') and a.cnt >= 10 then 1 else 0 end as user_cat1_flag, 
case when a.domain in ('zakon.kz', 'egov.kz', 'makler.md') and a.cnt >= 10 then 1 else 0 end as user_cat2_flag,
case when a.domain in ('russianfood.com', 'psychologies.ru', 'gotovim.ru') and a.cnt >= 10 then 1 else 0 end as user_cat3_flag,
case when a.domain in ('books.imhonet.ru', 'zhurnaly.biz', 'zvukobook.ru') and a.cnt >= 10 then 1 else 0 end as user_cat4_flag from 
(select uid, domain, count(*) as cnt from raw_data group by uid, domain) a
order by a.uid;

In [ ]:
hdfs dfs -cat /user/valeria.lupanova/lab03/output/* > ~/lab03_users.txt

In [ ]:
INSERT OVERWRITE DIRECTORY 'hdfs://bd-master.newprolab.com:8020/user/valeria.lupanova/lab03s/input'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
select a.domain, nvl(b.user_cat1_flag,0) 
from raw_data a left join valeria_lupanova b 
on a.uid = b.uid 
and b.user_cat1_flag=1 
order by a.domain;

In [ ]:
create external table valeria_lupanova.raw_data_s
(domain string comment 'Экранированный URL',
is_autouser string,
is_total)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
LOCATION 'hdfs://bd-master.newprolab.com:8020/user/valeria.lupanova/lab03s/input';

In [ ]:
select distinct domain
from raw_data_s;

In [ ]:
select domain,
sum(is_autouser) as current_domains_with_1
from raw_data_s
group by domain;

In [ ]:
select domain,
count(domain) as current_domains
from raw_data_s
group by domain;

In [ ]:
select 
sum(is_autouser) as total_domains_with_1,
sum(case when is_autouser = 0 then 1 else is_autouser end) as total_domains 
from raw_data_s;

In [ ]:
INSERT OVERWRITE DIRECTORY 'hdfs://bd-master.newprolab.com:8020/user/valeria.lupanova/lab03s/output'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
select a0.domain, 
round((pow(a3.current_domains_with_1/a1.total_domains,2))/
      ((a2.current_domains/a1.total_domains)*(a1.total_domains_with_1/a1.total_domains)),20) as relevance
from (select distinct domain
from raw_data_s) a0 
join (select 
sum(is_autouser) as total_domains_with_1,
sum(case when is_autouser = 0 then 1 else is_autouser end) as total_domains 
from raw_data_s) a1
join (select domain,
count(domain) as current_domains
from raw_data_s
group by domain) a2 on a0.domain = a2.domain
join (select domain,
sum(is_autouser) as current_domains_with_1
from raw_data_s
group by domain) a3 on a0.domain = a3.domain
order by 2 desc, 1 asc
limit 200;

In [ ]:
hdfs dfs -cat /user/valeria.lupanova/lab03s/output/* > ~/lab03s_domains.txt

Declaring libraries =>

In [ ]:
#!/opt/anaconda/envs/bd9/bin/python

from urllib.parse import urlparse, unquote
import sys
import re

Declaring url2domain() for getting domain =>

In [ ]:
def url2domain(url):
    try:
        a = urlparse(unquote(url.strip()))
        if (a.scheme in ['http','https']):
            b = re.search("(?:www\.)?(.*)",a.netloc).group(1)
            if b is not None:
                return str(b).strip()
            else:
                return ''
        else:
            return ''
    except:
        return

Lab variables =>

In [ ]:
# User categories
category_1 = 'Autouser'
category_2 = 'Businessuser'
category_3 = 'Homeuser'
category_4 = 'Bookuser'
# Analysed domains 
domains_category_1 = [u'cars.ru', u'avto-russia.ru', u'bmwclub.ru']
domains_category_2 = [u'zakon.kz', u'egov.kz', u'makler.md']
domains_category_3 = [u'russianfood.com', u'psychologies.ru', u'gotovim.ru']
domains_category_4 = [u'books.imhonet.ru', u'zhurnaly.biz', u'zvukobook.ru']

# User identificators by categories
id1 = 2
id2 = 3
id3 = 4
id4 = 5

Declaring mapper() =>

In [ ]:
def mapper(line):
    uid, ts, url = line.split('\t')
    if (re.match(r'^[\d]+$', (uid.split('\t'))[0]) and re.match(r'^[\d]+\.[\d]+$', ts)
        and (re.match(r'^http.+$', url) or re.match(r'^https.+$', url))):
        print('{}\t{}\t{}\n'.format(uid, ts, url))

Declaring main() =>

In [ ]:
def main():
    try:
        for line in sys.stdin:
            mapper(line)
    except:
        return

Declaring __name__ method =>

In [ ]:
if __init == '__main':
    def main()